In [19]:
!pip install gspread
!pip install scrapydo

In [20]:
!pip install google-auth

In [1]:
import scrapy
import scrapydo
import gspread
from google.auth.transport.requests import Request
from google.auth import exceptions
from google.oauth2 import service_account

class LinkedJobsSpider(scrapy.Spider):
    name = "linkedin_jobs"
    api_url = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=DataAnalyst&location=India&locationId=&geoId=102713980&f_TPR=&f_JT=F%2CC&f_WT=2&start=' 
    google_sheet_key = '1hmtmEUKbCuBcQzZ-zy8AEr5w9BrtZMAql3-ZDqtT1_Y'  #Google Sheet key
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

    def start_requests(self):
        first_job_on_page = 0
        first_url = self.api_url + str(first_job_on_page)
        yield scrapy.Request(url=first_url, callback=self.parse_job, meta={'first_job_on_page': first_job_on_page, 'seen_records': set()})

    def parse_job(self, response):
        first_job_on_page = response.meta['first_job_on_page']
        seen_records = response.meta['seen_records']

        jobs = response.css("li")

        num_jobs_returned = len(jobs)
        print("******* Num Jobs Returned *******")
        print(num_jobs_returned)
        print('*****')
        
        for job in jobs:
            job_item = {
                'job_title': job.css("h3::text").get(default='not-found').strip(),
                'job_detail_url': job.css(".base-card__full-link::attr(href)").get(default='not-found').strip(),
                'job_listed': job.css('time::text').get(default='not-found').strip(),
                'company_name': job.css('h4 a::text').get(default='not-found').strip(),
                'company_link': job.css('h4 a::attr(href)').get(default='not-found'),
                'company_location': job.css('.job-search-card__location::text').get(default='not-found').strip(),
            }

            # Create a unique identifier for the record
            unique_identifier = (
                job_item['company_name'],
                job_item['company_location'],
                job_item['job_title'],
                job_item['job_detail_url']
            )

            # Check if the record is unique
            if all(unique_identifier) and unique_identifier not in seen_records:
                # Print job details
                print("******* Job Details *******")
                print(job_item)

                # Sync to Google Sheet
                self.sync_to_google_sheet(job_item, seen_records)

                # Yield the unique record
                seen_records.add(unique_identifier)
                yield job_item

        # REQUEST NEXT PAGE OF JOBS HERE
        if num_jobs_returned > 0:
            next_page_number = int(first_job_on_page) + 25
            next_url = self.api_url + str(next_page_number)
            yield scrapy.Request(url=next_url, callback=self.parse_job, meta={'first_job_on_page': next_page_number, 'seen_records': seen_records})

    def sync_to_google_sheet(self, data, seen_records):
        creds_file_path = '/Downloads/apis.json' #replace this with json file format

        try:
            creds = service_account.Credentials.from_service_account_file(creds_file_path, scopes=self.scope)
            creds.refresh(Request())
        except exceptions.GoogleAuthError as e:
            print(f"Error authenticating with Google Sheets API: {e}")
            raise SystemExit(e)

        # Authenticate with Google Sheets API
        gc = gspread.authorize(creds)

        # Open the Google Sheet by key
        sheet = gc.open_by_key(self.google_sheet_key)

        # Select the first worksheet
        worksheet = sheet.get_worksheet(0)

        # Create a unique identifier for the record
        unique_identifier = (
            data['company_name'],
            data['company_location'],
            data['job_title'],
            data['job_detail_url']
        )

        # Check if the record already exists in the Google Sheet
        if unique_identifier not in seen_records:
            # Append data to the worksheet
            worksheet.append_row([data['job_title'], data['job_detail_url'], data['job_listed'],
                                data['company_name'], data['company_link'], data['company_location']])
            print("Data successfully appended to Google Sheet")

            # Add the unique identifier to the set of seen records
            seen_records.add(unique_identifier)
        else:
            print("Data already exists in Google Sheet")




In [2]:
# Setup scrapydo
scrapydo.setup()

# Run the spider
scrapydo.run_spider(LinkedJobsSpider)

******* Num Jobs Returned *******
25
*****
******* Job Details *******
{'job_title': 'Data Analyst / Analysis- (Fresher Batch)', 'job_detail_url': 'https://in.linkedin.com/jobs/view/data-analyst-analysis-fresher-batch-at-razormet-technologies-3777097888?refId=Js1qWSIwqWzMpzlpLfwRIw%3D%3D&trackingId=RCK9c%2Bh%2FNg9PZj4vfcwoBw%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card', 'job_listed': '3 weeks ago', 'company_name': 'Razormet Technologies', 'company_link': 'https://in.linkedin.com/company/razormet?trk=public_jobs_jserp-result_job-search-card-subtitle', 'company_location': 'India'}
Data successfully appended to Google Sheet
******* Job Details *******
{'job_title': 'Lead Data Analyst', 'job_detail_url': 'https://in.linkedin.com/jobs/view/lead-data-analyst-at-hypersonix-inc-3782173920?refId=Js1qWSIwqWzMpzlpLfwRIw%3D%3D&trackingId=GVrSmeaZ7Ke5CO3hayC3Zw%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card', 'job_listed': '1 day ago', 'company_name': 'H

[{'job_title': 'Data Analyst / Analysis- (Fresher Batch)',
  'job_detail_url': 'https://in.linkedin.com/jobs/view/data-analyst-analysis-fresher-batch-at-razormet-technologies-3777097888?refId=Js1qWSIwqWzMpzlpLfwRIw%3D%3D&trackingId=RCK9c%2Bh%2FNg9PZj4vfcwoBw%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card',
  'job_listed': '3 weeks ago',
  'company_name': 'Razormet Technologies',
  'company_link': 'https://in.linkedin.com/company/razormet?trk=public_jobs_jserp-result_job-search-card-subtitle',
  'company_location': 'India'},
 {'job_title': 'Lead Data Analyst',
  'job_detail_url': 'https://in.linkedin.com/jobs/view/lead-data-analyst-at-hypersonix-inc-3782173920?refId=Js1qWSIwqWzMpzlpLfwRIw%3D%3D&trackingId=GVrSmeaZ7Ke5CO3hayC3Zw%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card',
  'job_listed': '1 day ago',
  'company_name': 'Hypersonix Inc.',
  'company_link': 'https://www.linkedin.com/company/hypersonix-ai?trk=public_jobs_jserp-result_job-search-